# Triton Spark TTS + RVC Voice Conversion (Unified Container)

This notebook runs the full pipeline with **both services inside a single udocker container**:
1. **Triton Spark TTS** - Text-to-Speech (internal gRPC, port 8001)
2. **RVC Voice Conversion** - Voice cloning (inline processing)
3. **Voice HTTP API** - Unified endpoint (port 8003)

## Architecture
```
┌─────────────────────────────────────────────────────────┐
│  udocker Container                                      │
│  ┌─────────────────┐    ┌─────────────────────────────┐ │
│  │ Triton Server   │◄───│ Voice HTTP API (:8003)      │ │
│  │ (Spark TTS)     │    │ - FastAPI wrapper           │ │
│  │ :8001 gRPC      │    │ - TTS + RVC combined        │ │
│  │ (internal)      │    │ - N parallel RVC workers    │ │
│  └─────────────────┘    └─────────────────────────────┘ │
└─────────────────────────────────────────────────────────┘
                                   ▲
                                   │ HTTP
                                   ▼
                            Host Python
                         (simple requests)
```

## Port Allocation
- `8000` - Triton HTTP (internal)
- `8001` - Triton gRPC (internal)
- `8002` - Triton Metrics (internal)
- `8003` - Voice HTTP API (external)

## API Endpoints
- `POST /synthesize` - Full TTS + RVC pipeline
- `POST /synthesize/stream` - Streaming audio response
- `POST /tts` - TTS only
- `POST /rvc` - RVC only
- `GET /health` - Health check
- `GET /status` - Detailed status

## Benefits
- Simple HTTP interface (no gRPC complexity on host)
- Both services share CUDA context inside container
- Single container to manage
- No GPU memory fragmentation

## Requirements
- Google Colab with GPU runtime (T4 minimum, A100 recommended)
- ~12GB disk space for models and dependencies

---
## Section 1: Setup Environment

In [ ]:
# Cell 1.1: Initialize udocker
import os

def udocker_init():
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print('udocker initialized')

udocker_init()

In [ ]:
# Cell 1.2: Clone repository
!git clone https://github.com/VSlobolinskyi/triton-spark-server.git
%cd triton-spark-server
!git checkout feature/rvc-integration

In [ ]:
# Cell 1.3: GPU Configuration for udocker
import subprocess

def detect_gpu_paths():
    paths = {}
    paths['nvidia_smi'] = subprocess.getoutput('which nvidia-smi')
    paths['cuda_dir'] = subprocess.getoutput("find /usr -path '*/cuda*' -type d -maxdepth 3 | grep -v 'targets' | head -1")
    paths['nvidia_lib_dir'] = subprocess.getoutput("find /usr -name 'libcuda.so*' -o -name 'libnvidia-ml.so*' | grep -v 'stubs' | xargs dirname | sort -u | head -1")
    paths['ld_library_path'] = f"{paths['nvidia_lib_dir']}:{paths['cuda_dir']}/lib64:{paths['cuda_dir']}/compat"

    basic_devices = ['/dev/nvidia0', '/dev/nvidiactl', '/dev/nvidia-uvm']

    volumes = [
        f"--volume={paths['nvidia_smi']}:{paths['nvidia_smi']}",
        f"--volume={paths['nvidia_lib_dir']}:{paths['nvidia_lib_dir']}",
        f"--volume={paths['cuda_dir']}:{paths['cuda_dir']}"
    ]

    for device in basic_devices:
        if os.path.exists(device):
            volumes.append(f"--volume={device}:{device}")

    env_vars = [
        f"--env=LD_LIBRARY_PATH={paths['ld_library_path']}:/usr/local/tensorrt/targets/x86_64-linux-gnu/lib:$LD_LIBRARY_PATH",
        f"--env=PATH=/opt/tritonserver/bin:{paths['cuda_dir']}/bin:/usr/bin:$PATH",
        "--env=NVIDIA_VISIBLE_DEVICES=all",
        "--env=NVIDIA_DRIVER_CAPABILITIES=compute,utility"
    ]

    return {
        'paths': paths,
        'volumes': volumes,
        'env_vars': env_vars,
        'all_options': ' '.join(volumes + env_vars)
    }

gpu_config = detect_gpu_paths()
print("GPU configuration ready")
for k, v in gpu_config['paths'].items():
    print(f"  {k}: {v}")

---
## Section 2: Setup Container with TTS + RVC

In [ ]:
# Cell 2.1: Pull Triton Server image
!udocker --allow-root pull nvcr.io/nvidia/tritonserver:25.02-trtllm-python-py3
!udocker --allow-root create --name=voice_server nvcr.io/nvidia/tritonserver:25.02-trtllm-python-py3
!udocker --allow-root setup --nvidia voice_server

In [ ]:
# Cell 2.2: Install TTS dependencies (torchaudio, etc.)
import os
pwd = os.getcwd()

tts_deps_cmd = f'''udocker --allow-root run \
    --hostauth --hostenv \
    {gpu_config['all_options']} \
    --volume={pwd}:/workspace \
    voice_server \
    /bin/bash -c "apt-get update && apt-get install -y cmake curl ffmpeg && \
git clone https://github.com/pytorch/audio.git && cd audio && git checkout c670ad8 && USE_FFMPEG=0 PATH=/usr/local/cuda/bin:\$PATH python3 setup.py develop && \
pip install einx==0.3.0 omegaconf==2.3.0 soundfile==0.12.1 soxr==0.5.0.post1 tritonclient librosa 'huggingface-hub>=0.24.0,<1.0'"
'''

!{tts_deps_cmd}

In [ ]:
# Cell 2.3: Install RVC + HTTP API dependencies inside container
import os
pwd = os.getcwd()

rvc_deps_cmd = f'''udocker --allow-root run \
    --hostauth --hostenv \
    {gpu_config['all_options']} \
    --volume={pwd}:/workspace \
    voice_server \
    /bin/bash -c "pip install --no-cache-dir \
        faiss-cpu>=1.7.0 \
        pyworld==0.3.2 \
        praat-parselmouth>=0.4.2 \
        torchcrepe==0.0.23 \
        ffmpeg-python>=0.2.0 \
        av>=9.0.0 \
        onnxruntime-gpu>=1.13.0 \
        numba>=0.56.0 \
        joblib>=1.1.0 \
        fastapi>=0.100.0 \
        uvicorn>=0.21.1 \
        python-multipart>=0.0.6 && \
    pip install --no-cache-dir git+https://github.com/One-sixth/fairseq.git"
'''

print("Installing RVC + HTTP API dependencies in container...")
!{rvc_deps_cmd}

In [ ]:
# Cell 2.4: Build TensorRT-LLM engine for Spark TTS
import os
pwd = os.getcwd()

build_cmd = f'''udocker --allow-root run \
    --hostauth --hostenv \
    {gpu_config['all_options']} \
    --env=PYTHONPATH=/workspace \
    --volume={pwd}:/workspace \
    voice_server \
    /bin/bash -c "cd /workspace/triton && bash run.sh 0 2 offline"'''

print("Building TensorRT engine for Spark TTS...")
!{build_cmd}

---
## Section 3: Download Models

In [ ]:
# Cell 3.1: Download RVC assets (HuBERT, RMVPE) - run inside container
import os
pwd = os.getcwd()

assets_cmd = f'''udocker --allow-root run \
    --hostauth --hostenv \
    {gpu_config['all_options']} \
    --env=PYTHONPATH=/workspace \
    --volume={pwd}:/workspace \
    voice_server \
    /bin/bash -c "cd /workspace && python3 tools/download_all_assets.py --rvc-only"'''

print("Downloading RVC assets...")
!{assets_cmd}

In [ ]:
# Cell 3.2: Download RVC voice model
import os
pwd = os.getcwd()

RVC_MODEL_URL = "https://huggingface.co/Juneuarie/SilverWolfEN/resolve/main/SilverWolf.zip?download=true"

model_cmd = f'''udocker --allow-root run \
    --hostauth --hostenv \
    {gpu_config['all_options']} \
    --env=PYTHONPATH=/workspace \
    --volume={pwd}:/workspace \
    voice_server \
    /bin/bash -c "cd /workspace && python3 tools/download_all_assets.py --rvc-model '{RVC_MODEL_URL}'"'''

print("Downloading RVC voice model...")
!{model_cmd}

# List downloaded models and index files
print("\n=== Downloaded Files ===")
print("\nVoice models (.pth):")
!ls -la assets/weights/*.pth 2>/dev/null || echo "  No .pth files found in assets/weights/"

print("\nIndex files (.index) - REQUIRED for voice quality:")
!ls -la logs/*.index 2>/dev/null || echo "  WARNING: No .index files found in logs/"
!ls -la assets/*.index 2>/dev/null || echo ""

# Check if index exists
import glob
index_files = glob.glob("logs/*.index") + glob.glob("assets/*.index")
if not index_files:
    print("\n*** WARNING: No index file found! Voice quality enhancement will be disabled. ***")
    print("The RVC model zip should contain a .index file for best results.")

---
## Section 4: Start Voice Services

In [ ]:
# Cell 4.1: Start Voice HTTP API (Triton + RVC inside container)
import subprocess
import time
import os

pwd = os.getcwd()

# Configuration
RVC_MODEL = "SilverWolf_e300_s6600.pth"
RVC_WORKERS = 2  # 2 for T4, up to 4 for A100
API_PORT = 8003  # Triton uses 8000-8002, so we use 8003

# Start services via startup script
# NOTE: RVC requires these environment variables for model loading:
#   - rmvpe_root: Path to rmvpe.pt for pitch extraction
#   - hubert_base_path: Path to hubert_base.pt for feature extraction
#   - weight_root: Path to RVC voice model weights
#   - index_root: Path to .index files for voice quality enhancement
server_cmd = f'''udocker --allow-root run \
    --hostauth --hostenv \
    {gpu_config['all_options']} \
    --env=PYTHONPATH=/workspace \
    --env=RVC_MODEL={RVC_MODEL} \
    --env=RVC_WORKERS={RVC_WORKERS} \
    --env=API_PORT={API_PORT} \
    --env=rmvpe_root=/workspace/assets/rmvpe \
    --env=hubert_base_path=/workspace/assets/hubert/hubert_base.pt \
    --env=weight_root=/workspace/assets/weights \
    --env=index_root=/workspace/logs \
    --volume={pwd}:/workspace \
    voice_server \
    /bin/bash /workspace/scripts/start_services.sh'''

# Run in background
process = subprocess.Popen(
    f"nohup {server_cmd} > services_log.txt 2>&1 &",
    shell=True
)

print("Starting voice services...")
print(f"  Voice HTTP API: http://localhost:{API_PORT}")
print(f"  RVC Workers: {RVC_WORKERS}")
print(f"  RVC Model: {RVC_MODEL}")
print("\nWaiting 150 seconds for services to initialize...")
time.sleep(150)

print("\nService logs:")
!tail -50 services_log.txt

In [ ]:
# Cell 4.2: Install host Python dependencies (just requests + audio)
!pip install -q requests soundfile numpy

print("Host dependencies installed")

In [ ]:
# Cell 4.3: Test connection to Voice HTTP API
import requests

API_URL = "http://localhost:8003"

print("Testing Voice HTTP API connection...")

# Health check
try:
    response = requests.get(f"{API_URL}/health", timeout=10)
    if response.status_code == 200:
        health = response.json()
        print(f"  Health: {health['status']}")
        print(f"  Triton ready: {health.get('triton_ready', 'N/A')}")
        print(f"  RVC ready: {health.get('rvc_ready', 'N/A')}")
    else:
        print(f"  Health check failed: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"  Connection error: {e}")

# Detailed status
print("\nDetailed status:")
try:
    response = requests.get(f"{API_URL}/status", timeout=10)
    if response.status_code == 200:
        status = response.json()
        print(f"  RVC Model: {status.get('rvc_model', 'N/A')}")
        print(f"  RVC Workers: {status.get('rvc_workers', 'N/A')}")
        print(f"  Triton address: {status.get('triton_addr', 'N/A')}:{status.get('triton_port', 'N/A')}")
except requests.exceptions.RequestException as e:
    print(f"  Status check failed: {e}")

print("\nGPU memory usage:")
!nvidia-smi --query-gpu=memory.used,memory.total --format=csv

---
## Section 5: Upload Reference Audio

In [ ]:
# Cell 5.1: Upload reference audio
import os
import soundfile as sf
from google.colab import files

!mkdir -p references TEMP/output

print("Please upload a reference audio file (.wav format):")
uploaded = files.upload()

reference_audio = list(uploaded.keys())[0]
references_path = os.path.join("references", reference_audio)

with open(references_path, "wb") as f:
    f.write(uploaded[reference_audio])

audio, sr = sf.read(references_path)
print(f"Saved: {references_path} ({len(audio)/sr:.2f}s @ {sr}Hz)")

# Resample to 16kHz if needed
if sr != 16000:
    base_name = os.path.splitext(reference_audio)[0]
    reference_audio_16k = os.path.join("references", f"{base_name}_16k.wav")
    !ffmpeg -y -i "{references_path}" -ar 16000 "{reference_audio_16k}" -loglevel error
    print(f"Resampled to 16kHz: {reference_audio_16k}")
else:
    reference_audio_16k = references_path

---
## Section 6: Run Voice Synthesis

In [ ]:
# Cell 6.1: Synthesize with HTTP API (full pipeline)
import requests
import time
import io

API_URL = "http://localhost:8003"

# Configuration
TEXT = "Hello! This is a test of the unified voice synthesis pipeline. Both TTS and RVC are running inside a single container, exposed via a simple HTTP API."
REFERENCE_TEXT = ""
PITCH_SHIFT = 0
F0_METHOD = "rmvpe"
INDEX_RATE = 1.0  # Use 1.0 for maximum quality enhancement

print(f"Text: {TEXT[:60]}...")
print(f"Reference: {reference_audio_16k}")
print(f"Settings: pitch={PITCH_SHIFT}, f0={F0_METHOD}, index_rate={INDEX_RATE}")

# Prepare request
with open(reference_audio_16k, "rb") as f:
    ref_audio_data = f.read()

# Call synthesize endpoint
print("\nRunning TTS + RVC synthesis...")
start_time = time.time()

response = requests.post(
    f"{API_URL}/synthesize",
    data={
        "text": TEXT,
        "reference_text": REFERENCE_TEXT,
        "pitch_shift": PITCH_SHIFT,
        "f0_method": F0_METHOD,
        "index_rate": INDEX_RATE,
        "skip_rvc": False,
    },
    files={
        "reference_audio": ("reference.wav", ref_audio_data, "audio/wav")
    },
    timeout=120
)

total_time = time.time() - start_time

if response.status_code == 200:
    # Get timing from headers
    tts_time = float(response.headers.get("X-TTS-Time", 0))
    rvc_time = float(response.headers.get("X-RVC-Time", 0))
    
    # Save audio
    output_path = "TEMP/output/synthesized.wav"
    with open(output_path, "wb") as f:
        f.write(response.content)
    
    # Get duration
    audio, sr = sf.read(output_path)
    duration = len(audio) / sr
    
    print(f"\nSuccess!")
    print(f"  TTS time: {tts_time:.2f}s")
    print(f"  RVC time: {rvc_time:.2f}s")
    print(f"  Total time: {total_time:.2f}s")
    print(f"  Audio duration: {duration:.2f}s")
    print(f"  Saved to: {output_path}")
else:
    print(f"\nError: {response.status_code}")
    print(response.text)

In [ ]:
# Cell 6.2: Play output audio
from IPython.display import Audio, display
import os

output_path = "TEMP/output/synthesized.wav"

if os.path.exists(output_path):
    print("Synthesized audio (TTS + RVC):")
    display(Audio(output_path))
else:
    print("No output file found. Run synthesis first.")

In [ ]:
# Cell 6.3: TTS only (skip RVC)
import requests
import time

API_URL = "http://localhost:8003"

TEXT = "This is TTS only, without voice conversion."

print(f"Text: {TEXT}")
print("Running TTS only (skip_rvc=True)...")

with open(reference_audio_16k, "rb") as f:
    ref_audio_data = f.read()

start_time = time.time()

response = requests.post(
    f"{API_URL}/synthesize",
    data={
        "text": TEXT,
        "reference_text": "",
        "skip_rvc": True,  # Skip RVC
    },
    files={
        "reference_audio": ("reference.wav", ref_audio_data, "audio/wav")
    },
    timeout=60
)

if response.status_code == 200:
    tts_time = float(response.headers.get("X-TTS-Time", 0))
    
    output_path = "TEMP/output/tts_only.wav"
    with open(output_path, "wb") as f:
        f.write(response.content)
    
    print(f"TTS time: {tts_time:.2f}s")
    print(f"Saved to: {output_path}")
    
    from IPython.display import Audio, display
    display(Audio(output_path))
else:
    print(f"Error: {response.status_code} - {response.text}")

---
## Section 7: Batch Processing

In [ ]:
# Cell 7.1: Process multiple sentences
import requests
import time
import re
import soundfile as sf

API_URL = "http://localhost:8003"

LONG_TEXT = """
This is the first sentence of our test.
The second sentence demonstrates the HTTP API approach.
Third sentence continues the voice synthesis test.
Fourth sentence shows the unified container working.
Fifth and final sentence completes our batch test.
""".strip()

# Split into sentences
sentences = [s.strip() for s in re.split(r'(?<=[.!?])\s+', LONG_TEXT) if s.strip()]
print(f"Processing {len(sentences)} sentences...\n")

# Load reference audio once
with open(reference_audio_16k, "rb") as f:
    ref_audio_data = f.read()

total_start = time.time()
results = []

for i, sentence in enumerate(sentences):
    print(f"[{i+1}/{len(sentences)}] {sentence[:40]}...")
    
    start = time.time()
    response = requests.post(
        f"{API_URL}/synthesize",
        data={
            "text": sentence,
            "reference_text": "",
            "pitch_shift": PITCH_SHIFT,
            "f0_method": F0_METHOD,
            "index_rate": INDEX_RATE,
        },
        files={
            "reference_audio": ("reference.wav", ref_audio_data, "audio/wav")
        },
        timeout=120
    )
    elapsed = time.time() - start
    
    if response.status_code == 200:
        tts_time = float(response.headers.get("X-TTS-Time", 0))
        rvc_time = float(response.headers.get("X-RVC-Time", 0))
        
        output_path = f"TEMP/output/sentence_{i}.wav"
        with open(output_path, "wb") as f:
            f.write(response.content)
        
        audio, sr = sf.read(output_path)
        results.append(audio)
        
        print(f"    TTS: {tts_time:.2f}s, RVC: {rvc_time:.2f}s, Total: {elapsed:.2f}s")
    else:
        print(f"    Error: {response.status_code}")

total_time = time.time() - total_start
print(f"\nTotal: {total_time:.2f}s for {len(sentences)} sentences ({total_time/len(sentences):.2f}s/sentence)")

In [ ]:
# Cell 7.2: Concatenate and play all sentences
import numpy as np
import soundfile as sf
from IPython.display import Audio, display

if results:
    # Add small silence between sentences
    silence = np.zeros(int(16000 * 0.3))  # 300ms
    
    combined = []
    for i, audio in enumerate(results):
        combined.append(audio)
        if i < len(results) - 1:
            combined.append(silence)
    
    full_audio = np.concatenate(combined)
    output_path = "TEMP/output/combined.wav"
    sf.write(output_path, full_audio, 16000)
    
    print(f"Combined audio: {len(full_audio)/16000:.2f}s")
    display(Audio(output_path))
else:
    print("No results to combine.")

---
## Section 8: Advanced Options

In [ ]:
# Cell 8.1: RVC only (convert existing audio)
import requests
import soundfile as sf
from IPython.display import Audio, display

API_URL = "http://localhost:8003"

# Use TTS output as input for RVC-only conversion
input_audio = "TEMP/output/tts_only.wav"

if os.path.exists(input_audio):
    print(f"Converting: {input_audio}")
    
    with open(input_audio, "rb") as f:
        audio_data = f.read()
    
    response = requests.post(
        f"{API_URL}/rvc",
        data={
            "pitch_shift": 0,
            "f0_method": "rmvpe",
            "index_rate": 1.0,
        },
        files={
            "audio": ("input.wav", audio_data, "audio/wav")
        },
        timeout=60
    )
    
    if response.status_code == 200:
        output_path = "TEMP/output/rvc_only.wav"
        with open(output_path, "wb") as f:
            f.write(response.content)
        
        rvc_time = float(response.headers.get("X-RVC-Time", 0))
        print(f"RVC time: {rvc_time:.2f}s")
        print(f"Saved to: {output_path}")
        
        print("\nOriginal:")
        display(Audio(input_audio))
        print("\nRVC converted:")
        display(Audio(output_path))
    else:
        print(f"Error: {response.status_code} - {response.text}")
else:
    print(f"Input file not found: {input_audio}")
    print("Run Cell 6.3 (TTS only) first.")

In [ ]:
# Cell 8.2: Test different pitch shifts
import requests
import soundfile as sf
from IPython.display import Audio, display

API_URL = "http://localhost:8003"

TEXT = "Testing different pitch shifts."
PITCH_SHIFTS = [-6, 0, 6, 12]

with open(reference_audio_16k, "rb") as f:
    ref_audio_data = f.read()

for pitch in PITCH_SHIFTS:
    print(f"\nPitch shift: {pitch:+d} semitones")
    
    response = requests.post(
        f"{API_URL}/synthesize",
        data={
            "text": TEXT,
            "reference_text": "",
            "pitch_shift": pitch,
            "f0_method": "rmvpe",
            "index_rate": 1.0,
        },
        files={
            "reference_audio": ("reference.wav", ref_audio_data, "audio/wav")
        },
        timeout=60
    )
    
    if response.status_code == 200:
        output_path = f"TEMP/output/pitch_{pitch:+d}.wav"
        with open(output_path, "wb") as f:
            f.write(response.content)
        display(Audio(output_path))
    else:
        print(f"  Error: {response.status_code}")

---
## Section 9: Shutdown

In [ ]:
# Cell 9.1: Shutdown services
import time

print("Shutting down services...")

# Kill processes inside container
!pkill -f tritonserver
!pkill -f "uvicorn.*voice_api"
!pkill -f "udocker.*voice_server"

time.sleep(3)

print("\nGPU status after shutdown:")
!nvidia-smi